# Section 8.2: Time Matrix Multiply

Adapted from: "Guide to Fortran 2008 Programming" by Walter S. Brainerd (Springer 2015)

## Program to demonstrate the performance improvement of using matmul over do loops.

```fortran
program time_matrix_multiply

    ! Compare times of the matmul intrinsic cs DO loops

    implicit none
    integer, parameter :: n = 10
    real, dimension (n, n) :: a, b, c1, c2
    character(len=8) :: date
    real :: start_time_1, stop_time_1, start_time_2, stop_time_2
    real :: total_time_1, total_time_2
    integer :: i, j, k
    character(len=*), parameter :: form = "(t2, a, f0.10, a)"

    ! Get date to print on report

    call date_and_time(date = date)

    print *, "Timing report dated: " // date(1:4) &
        // "-" // date(5:6) // "-" // date(7:8)

    call random_seed()
    call random_number(a)
    call random_number(b)
    call cpu_time(start_time_1)
   
    ! Lines below added for effect
    ! The matrices of random values will be printed to the screen
    write (*, "(a)") "Matrix A"
    write (*, "(10f10.3)") (a(i,:), i=1,n)

    print *
    write (*, "(a)") "Matrix B"
    write (*, "(10f10.3)") (b(i,:), i=1,n)

    c1 = 0

    do k = 1, n
        do j = 1, n
            do i = 1, n
                c1(i, j) = c1(i, j) + a(i, k) * b(k, j)
            end do
        end do
    end do

    call cpu_time(stop_time_1)

    total_time_1 = stop_time_1 - start_time_1

    print *

    write (*, "(a)") "Matrix C1 is AxB using loops."
    write (*, "(10f10.3)") (c1(i,:), i=1,n)

    call cpu_time(start_time_2)
    c2 = matmul(a, b)
    call cpu_time(stop_time_2)

    total_time_2 = stop_time_2 - start_time_2

    print *

    write (*, "(a)") "Matrix C2 is AxB using matmul."
    write (*, "(10f10.3)") (c1(i,:), i=1,n)

    print *

    write (*, form) "Time of Do loop version is: ", total_time_1, " seconds."
    write (*, form) "Time of matmul version is: ", total_time_2, " seconds."

    print *

    if (any(abs(c1-c2) > 1.0e-4)) then
        write (*,  "(a)") "There are significantly different values between the matrices."
    else
        write (*,  "(a)") "The results are approximately the same."
    end if

    print *
    write (*, "(a, f10.3, a)") "The speedup ratio is: ", total_time_1/total_time_2, "x"

end program time_matrix_multiply
```

The above program is compiled and run using Fortran Package Manager (fpm):

In [1]:
import os
root_dir = os.getcwd()

In [2]:
code_dir = root_dir + "/" + "Fortran_Code/Section_8_2_Time_Matrix_Multiply"

In [3]:
os.chdir(code_dir)

In [4]:
build_status = os.system("fpm build > /dev/null")

fpm run output is piped into sed to suppress the status of the run command and only print the output of the executable.

In [5]:
exec_status = os.system("fpm run | sed 1,1d")

 Timing report dated: 2022-06-20
Matrix A
     0.808     0.715     0.145     0.883     0.607     0.526     0.897     0.725     0.061     0.935
     0.729     0.688     0.474     0.051     0.544     0.169     0.139     0.849     0.274     0.086
     0.379     0.576     0.636     0.147     0.614     0.729     0.820     0.541     0.412     0.735
     0.000     0.839     0.861     0.389     0.620     0.943     0.181     0.659     0.002     0.105
     0.643     0.622     0.307     0.152     0.227     0.079     0.712     0.725     0.120     0.006
     0.648     0.994     0.040     0.421     0.686     0.690     0.912     0.425     0.451     0.175
     0.997     0.998     0.799     0.750     0.976     0.176     0.065     0.567     0.295     0.966
     0.041     0.910     0.359     0.658     0.197     0.332     0.227     0.092     0.801     0.891
     0.338     0.712     0.350     0.681     0.237     0.273     0.572     0.092     0.732     0.744
     0.152     0.524     0.789     0.455     0.31

   1.407     2.228     2.035     3.024     2.705     2.979

 Time of Do loop version is: .0014549997 seconds.
 Time of matmul version is: .0000569997 seconds.

The results are approximately the same.

The speedup ratio is:     25.526x
